In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime
import math
import sympy

In [ ]:
#def theoretical_waiting_cdf_formula(lambd, mu1, mu2, perc, max_time):

#    f = 0.0
#    waiting_cdf = list()
#    for i in range(0, 200):
#        waiting = 1 - (1/16)*(np.exp(-(3/2)*lambd*f)) - (9/16)*(np.exp(-(1/2)*lambd*f))
#        waiting_cdf.append(waiting*100)
#        f = f+0.01

#    return waiting_cdf


In [ ]:
#def theoretical_system_cdf_formula(lambd, mu1, mu2, perc, max_time):

#    f = 0.0
#    system_cdf = list()
#    for i in range(0, 200):
#        system = 1 - (5/32)*(np.exp(-(3/2)*lambd*f)) - (27/32)*(np.exp(-(1/2)*lambd*f))
#        system_cdf.append(system*100)
#        f = f+0.01

#    return system_cdf


In [ ]:
def simulate_mm1_cdf(lambd, mu1, mu2, num_simulations, perc):
    waiting_times = []
    system_times = []
    arrival_tmp = 0
    arrival_times = []
    departure_times = []
    random.seed(datetime.now().timestamp())

    for i in range(num_simulations):
        interarrival_time = np.random.exponential(1/lambd)

        rr = random.randint(1,100)

        if rr <= perc*100:
            service_time = np.random.exponential(1/mu1)
        else:
            service_time = np.random.exponential(1/mu2)

        if i == 0:
            arrival_times.append(0)
        else:
            arrival_times.append(arrival_times[i-1] + interarrival_time)

        # Calculate waiting time
        if i == 0:
            waiting_time = 0
            waiting_times.append(waiting_time)
        else:
            waiting_time = max(0, departure_times[i-1] - arrival_times[i])
            waiting_times.append(waiting_time)
        # Calculate system time
        system_time = service_time + waiting_time
        system_times.append(system_time)

        if i == 0:
            departure_times.append(system_time)
        else:
            departure_times.append(arrival_times[i] + system_time)

    # Calculate CDFs
    sorted_waiting_times = np.sort(waiting_times)
    sorted_system_times = np.sort(system_times)

    simulation_wait_cdf = list()
    simulation_system_cdf = list()
    for i in range(0, 200):
        tt = 0.01*i
        wt = 0.0
        st = 0.0
        for j in range(0, num_simulations):
            if sorted_waiting_times[j] <= tt:
                wt += float(100/num_simulations)
            if sorted_system_times[j] <= tt:
                st += float(100/num_simulations)

        simulation_wait_cdf.append(wt)
        simulation_system_cdf.append(st)

    return simulation_wait_cdf, simulation_system_cdf

In [ ]:
def theoretical_wait_cdf_sympy(lambd, mu1, mu2,perc):
    s,x= sympy.symbols('s, x')
    L = sympy.symbols('L', real=True, positive=True)

    b=L*perc*sympy.exp(-L*x)+(1-perc)*2*L*sympy.exp(-2*L*x)

    B_star=sympy.symbols('B_star',cls=sympy.Function)
    B_star=sympy.laplace_transform(b,x,s)[0]

    rho=perc*float(1/(mu1/lambd))+(1-perc)*float(1/(mu2/lambd))

    W_star=s*(1-rho)/(s-L+L*B_star)

    w=sympy.inverse_laplace_transform(W_star, s, x)
    W=sympy.integrate(w,x)

    theoretical_waiting_cdf= list()
    f = 0.0
    for i in range(200):
        theoretical_waiting_cdf.append(W.subs({x:f,L:lambd})*100)
        f += 0.01

    return theoretical_waiting_cdf


In [ ]:
def theoretical_system_cdf_sympy(lambd, mu1, mu2,perc):
    s,x= sympy.symbols('s, x')
    L = sympy.symbols('L', real=True, positive=True)

    b=L*perc*sympy.exp(-L*x)+(1-perc)*2*L*sympy.exp(-2*L*x)

    B_star=sympy.symbols('B_star',cls=sympy.Function)
    B_star=sympy.laplace_transform(b,x,s)[0]

    rho=perc*float(1/(mu1/lambd))+(1-perc)*float(1/(mu2/lambd))

    S_star=B_star*s*(1-rho)/(s-L+L*B_star)

    sys=sympy.inverse_laplace_transform(S_star, s, x)
    S=sympy.integrate(sys,x)

    f = 0.0
    theoretical_system_cdf= list()
    for i in range(200):
        theoretical_system_cdf.append(S.subs({x:f,L:lambd})*100)
        f += 0.01

    return theoretical_system_cdf

In [ ]:
def plot_mm1_cdf(cdf, cdf2, max_time, cdf_label, lambd, mu):

    plt.plot(np.arange(0, max_time, 0.01), cdf, label="Theoritical", color="Red")
    plt.plot(np.arange(0, max_time, 0.01), cdf2, label='Simulation', color="Blue")
    plt.xlabel('y (Unit Time)')
    plt.ylabel('W(y) CDF(%)')
    plt.title('Scenerio I - ' + cdf_label + ' Time Distribution (λ='+str(float(lambd))+')')
    plt.yticks(np.arange(0, 110, 10))
    plt.grid(True)
    plt.xlim(0,2)
    plt.ylim(0, 110)
    plt.legend()
    plt.show()

In [ ]:
def run(arrival_rate, perc, max_time, num_simulations, case):
    service_rate1 = arrival_rate
    service_rate2 = 2*arrival_rate

    if case == "I":
      theoretical_waiting_cdf = theoretical_wait_cdf_sympy(arrival_rate, service_rate1, service_rate2, perc)
      theoretical_system_cdf = theoretical_system_cdf_sympy(arrival_rate, service_rate1, service_rate2, perc)
    else:
      theoretical_waiting_cdf = theoretical_wait_cdf_sympy(arrival_rate, service_rate1, service_rate2, perc)
      theoretical_system_cdf = theoretical_system_cdf_sympy(arrival_rate, service_rate1, service_rate2, perc)

    simulate_waiting_cdf, simulate_system_cdf = simulate_mm1_cdf(arrival_rate, service_rate1, service_rate2,num_simulations, perc)
    plot_mm1_cdf(theoretical_waiting_cdf ,simulate_waiting_cdf, 2, "Waiting", arrival_rate, service_rate1)
    print("Mean Squared Error:", np.mean((np.array(theoretical_waiting_cdf) - np.array(simulate_waiting_cdf))**2))
    plot_mm1_cdf(theoretical_system_cdf ,simulate_system_cdf, 2, "System", arrival_rate, service_rate1)
    print("Mean Squared Error:", np.mean((np.array(theoretical_system_cdf) - np.array(simulate_system_cdf))**2))


In [ ]:
if __name__ == "__main__":

    run(10,0.5,2,20000,"I")
    #run(10,0.25,2,20000,"II")